In [19]:
import os
import sys
sys.path.append('..')
sys.path.append('../..')
import argparse
import utils

from student_utils import *

In [56]:
import pandas as pd
import networkx as nx
import student_utils as s_utils
import utils

  Complete the following function.
======================================================================

In [71]:
"""
======================================================================
  Complete the following function.
======================================================================
"""

def solve(list_of_locations, list_of_homes, starting_car_location, adjacency_matrix, params=[]):
    """
    Write your algorithm here.
    Input:
        list_of_locations: A list of locations such that node i of the graph corresponds to name at index i of the list
        list_of_homes: A list of homes
        starting_car_location: The name of the starting location for the car
        adjacency_matrix: The adjacency matrix from the input file
    Output:
        A list of locations representing the car path
        A dictionary mapping drop-off location to a list of homes of TAs that got off at that particular location
        NOTE: both outputs should be in terms of indices not the names of the locations themselves
    """
    # Make networkx graph G
    #G = make_graph(adjacency_matrix)
    G = s_utils.adjacency_matrix_to_graph(adjacency_matrix)
    
    #Get adjacency dictionary of distances for each location
    adjacencies = make_dictionary(adjacency_matrix, list_of_locations)
    
    # Get indices of homes
    home_indices = [list_of_locations.index(i) for i in list_of_homes]
    
    #Get nodes
    nodes = make_nodes(adjacencies, list_of_homes, home_indices)
    node_homes = list(nodes.keys())
    
    node_paths, node_G = shortest_paths(G, list_of_locations, node_homes)
    
    #TSP on node_G
    
    

In [67]:
# Convert adjacency matrix -> pd DataFrame -> networkx Graph
def make_graph(adjacency_matrix):
    adj = adjacency_matrix
    
    df = pd.DataFrame(adj)
    df = df.replace("x", 0)
        
    G = nx.from_pandas_adjacency(df)
    G.name = 'Graph from pandas adjacency matrix'
    
    print(nx.info(G))
    
    return G

In [68]:
# Returns adjacency dictionary of distances for each location {location: [distances to every location], ...}

def make_dictionary(adjacency_matrix, locations):
    #Create dictionary for every location
    adjacencies = {}
    
    #Create dictionary of adjacencent locations for every location
    # adjacencies = {"loc" : [distance to every other loc], ...}
    # If distance == "x" -> None
    for i in range(0,len(locations)):
        adj = [None if j == "x" else j for j in adjacency_matrix[i]]
        adjacencies[locations[i]] = adj
    
    return adjacencies

In [69]:
"""TODO: Change nodes to locations instead of just homes""" 
# Returns dictionary of {node_home: [homes belonging to node], ...}

def make_nodes(adjacencies, homes, home_indices):    
    limit = 12000 #maximum distance away from node's base
    nodes = {} #create a node around every location
    
    #Create nodes for every home
    for home in homes:
        nodes[home] = [home] #Start every home's node with itself
        
        for index in home_indices:
            distance = adjacencies[home][index]
            if (distance != None) and (distance < limit):
                #append other home that is within limit to the node starting at that home
                current = nodes[home]
                nodes[home].append(homes[index]) #returns None
    
    #print(nodes)
    
    
    #Clean up node dictionary to only contain largest nodes ------------
    deleted_nodes = nodes.copy()
    homes_represented = homes
    nodes_to_keep = list()
    
    #for node in node.keys():
    
    while homes_represented:         
        v = list(deleted_nodes.values())
        k = list(deleted_nodes.keys())
        biggest_node = k[v.index(max(v, key=len))]
        
        #remove homes that are already included in list
        homes_represented = [x for x in homes_represented if x not in nodes[biggest_node]]
        
        for home in nodes[biggest_node]:
            deleted_nodes.pop(home, None)
        
        nodes_to_keep.append(biggest_node)
    
    #print(nodes_to_keep)
    
    return {key: nodes[key] for key in nodes_to_keep}

In [ ]:
#returns dictionary of shortest paths between nodes {(node_1, node_2): [list of path], ...} 
#    for outputting driver path
#returns new graph of just nodes and their associated distances
#    for TSP solving with dwave

def shortest_paths(G, list_of_locations, node_homes):
    node_paths = {}
    #node_distances = {}
    
    #Make new graph of nodes
    node_G = nx.Graph()
    
    for node in node_homes:
        node_G.add_node(node)
    
    #Get shortest path between every node and the distance of that path
    for node_s in node_homes:
        node_paths[node_s] = list()
        for node_t in node_homes:
            index_s = list_of_locations.index(node_s)
            index_t = list_of_locations.index(node_t)
            path = nx.shortest_path(G, source = index_s, target = index_t, weight = "weight")
            node_paths[(node_s, node_t)] = path
            
            path_weight = nx.shortest_path_length(G, source= index_s, target= index_t, weight= "weight")[
            node_G.add_edge(node_s, node_t, weight= path_weight)
    
    print(node_G.edges())
    
    return node_paths, node_G
    
    

In [ ]:
# Use dwave tsp.python

======================================================================
   No need to change any code below this line
======================================================================

In [9]:
"""
======================================================================
   No need to change any code below this line
======================================================================
"""

"""
Convert solution with path and dropoff_mapping in terms of indices
and write solution output in terms of names to path_to_file + file_number + '.out'

! dropoff_mapping = dicitonary of {dropoff_loc, [list of TAs dropped off]}
"""
def convertToFile(path, dropoff_mapping, path_to_file, list_locs):
    string = ''
    for node in path:
        string += list_locs[node] + ' '
    string = string.strip()
    string += '\n'

    dropoffNumber = len(dropoff_mapping.keys())
    string += str(dropoffNumber) + '\n'
    for dropoff in dropoff_mapping.keys():
        strDrop = list_locs[dropoff] + ' '
        for node in dropoff_mapping[dropoff]:
            strDrop += list_locs[node] + ' '
        strDrop = strDrop.strip()
        strDrop += '\n'
        string += strDrop
    utils.write_to_file(path_to_file, string)

def solve_from_file(input_file, output_directory, params=[]):
    print('Processing', input_file)

    input_data = utils.read_file(input_file)
    num_of_locations, num_houses, list_locations, list_houses, starting_car_location, adjacency_matrix = data_parser(input_data)
    car_path, drop_offs = solve(list_locations, list_houses, starting_car_location, adjacency_matrix, params=params)

    basename, filename = os.path.split(input_file)
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    output_file = utils.input_to_output(input_file, output_directory)

    convertToFile(car_path, drop_offs, output_file, list_locations)


def solve_all(input_directory, output_directory, params=[]):
    input_files = utils.get_files_with_extension(input_directory, 'in')

    for input_file in input_files:
        solve_from_file(input_file, output_directory, params=params)

In [6]:
if __name__=="__main__":
    parser = argparse.ArgumentParser(description='Parsing arguments')
    parser.add_argument('--all', action='store_true', help='If specified, the solver is run on all files in the input directory. Else, it is run on just the given input file')
    parser.add_argument('input', type=str, help='The path to the input file or directory')
    parser.add_argument('output_directory', type=str, nargs='?', default='.', help='The path to the directory where the output should be written')
    parser.add_argument('params', nargs=argparse.REMAINDER, help='Extra arguments passed in')
    args = parser.parse_args()
    output_directory = args.output_directory
    if args.all:
        input_directory = args.input
        solve_all(input_directory, output_directory, params=args.params)
    else:
        input_file = args.input
        solve_from_file(input_file, output_directory, params=args.params)

usage: ipykernel_launcher.py [-h] [--all] input [output_directory] ...
ipykernel_launcher.py: error: unrecognized arguments: -f


SystemExit: 2

In [72]:
solve_from_file("inputs/100_50.in", "outputs_imam")

Processing inputs/100_50.in
['Oakmere', 'Valhaven', 'Freyview', 'Newby', 'Ironshore', 'Butterbush', 'Morwald']


TypeError: cannot unpack non-iterable NoneType object